# Stage 2: Audio Encoder Training & Embedding Extraction

이 노트북은 **Stage 2 오디오 파이프라인**을 구현합니다.
1. **소형 멜-CNN 오디오 인코더**를 학습 (Contrastive Learning)
2. 학습된 인코더로 곡별 오디오 임베딩을 추출하여 `.npz`로 저장

**주요 흐름:**
- Item2Vec 모델과 Mel Spectrogram 파일의 교집합 곡 리스트 생성
- **등장 횟수 버킷 기반 계층 샘플링**으로 학습용 서브셋(50,000곡) 구성
- Contrastive Learning (InfoNCE Loss) 수행
- 학습된 모델로 임베딩 추출

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec  8 17:01:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.07                 Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   64C    P0             15W /   65W |     462MiB /   4096MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# [Cell 1] 환경 설정 & 기본 상수 정의

import os
import sys
import json
import random
import glob
import logging
import tarfile
import shutil
from typing import List, Dict, Tuple, Optional, Set
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from tqdm.auto import tqdm

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def set_seed(seed: int = 42):
    """재현성을 위한 난수 시드 고정"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# --- 상수 및 하이퍼파라미터 정의 ---

# Colab 여부 확인
IS_COLAB = 'google.colab' in sys.modules

# 경로 설정
if IS_COLAB:
    # Colab 환경 경로
    DRIVE_MOUNT_PATH = "/content/drive"
    DRIVE_TAR_DIR = "/content/drive/MyDrive/Colab Notebooks/melon-dataset"
    LOCAL_MEL_DIR = "/content/mel_data"
    ITEM2VEC_MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/models/v2_item2vec.model" # TODO: 수정 필요
    MELON_TAR_MAP_PATH = "/content/drive/MyDrive/Colab Notebooks/melon_tar_map.json" # TODO: 수정 필요
    STAGE2_EMB_OUTPUT_PATH = "/content/drive/MyDrive/Colab Notebooks/models/audio_embeddings_stage2.npz"
    TRAIN_JSON_PATH = "/content/drive/MyDrive/Colab Notebooks/melon-dataset/train.json" # TODO: 수정 필요
    SONG_META_PATH = "/content/drive/MyDrive/Colab Notebooks/melon-dataset/song_meta.json" # TODO: 수정 필요
    GENRE_JSON_PATH = "/content/drive/MyDrive/Colab Notebooks/melon-dataset/genre_gn_all.json" # TODO: 수정 필요
    SONG_STATS_CACHE_PATH = "/content/drive/MyDrive/path/to/song_stats_min2.parquet" # TODO: 수정 필요
else:
    # 로컬(Windows) 환경 경로 (recommend_model 폴더 기준)
    ITEM2VEC_MODEL_PATH = "../models/v2_item2vec.model"
    MELON_TAR_MAP_PATH = "../melon_tar_map.json"
    LOCAL_MEL_DIR = r"C:\Users\ASUS\data\arena_mel"  # 외부 데이터 경로 유지
    STAGE2_EMB_OUTPUT_PATH = "../models/audio_embeddings_stage2.npz"
    TRAIN_JSON_PATH = "../melon-dataset-excepttar/train.json"
    SONG_META_PATH = "../melon-dataset-excepttar/song_meta.json"
    GENRE_JSON_PATH = "../melon-dataset-excepttar/genre_gn_all.json"
    SONG_STATS_CACHE_PATH = "../song_stats_min2.parquet"

# 오디오 처리 관련
FIXED_T = 256

# 학습 하이퍼파라미터
BATCH_SIZE = 64
NUM_EPOCHS = 10
LEARNING_RATE = 1e-3
EMBED_DIM = 128
TEMPERATURE = 0.1
RANDOM_SEED = 42

# 버킷별 목표 샘플 개수 (총 50,000곡)
BUCKET_TARGET_COUNTS = {
    "B1": 20000,  # 1-5회 (40%)
    "B2": 10000,  # 6-10회 (20%)
    "B3": 7500,   # 11-20회 (15%)
    "B4": 6500,   # 21-50회 (13%)
    "B5": 6000    # 51+회 (12%)
}

assert sum(BUCKET_TARGET_COUNTS.values()) == 50000, "버킷 목표 합계가 50,000이 아닙니다!"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")
logger.info(f"Running in Colab: {IS_COLAB}")
set_seed(RANDOM_SEED)

In [11]:
# Drive에서 복원 (런타임 재시작 시)



if IS_COLAB:
    BACKUP_DIR = "/content/drive/MyDrive/Colab Notebooks/mel_data_backup"

    if os.path.exists(BACKUP_DIR):
        logger.info("📦 Restoring from Drive backup...")

        os.makedirs(LOCAL_MEL_DIR, exist_ok=True)

        # 🔥 메타데이터 먼저 확인
        backup_metadata = os.path.join(BACKUP_DIR, "extraction_metadata.json")

        if os.path.exists(backup_metadata):
            # 메타데이터 복원
            shutil.copy2(backup_metadata, os.path.join(LOCAL_MEL_DIR, "extraction_metadata.json"))

            # tar 폴더들 복원
            with open(backup_metadata, 'r') as f:
                metadata = json.load(f)

            restored = 0
            for tar_name in metadata.keys():
                tar_folder = tar_name.replace('.tar', '')
                src = os.path.join(BACKUP_DIR, tar_folder)
                dst = os.path.join(LOCAL_MEL_DIR, tar_folder)

                if os.path.exists(src) and not os.path.exists(dst):
                    shutil.copytree(src, dst)
                    restored += 1

            logger.info(f"✅ Restored {restored} tar folders")
        else:
            logger.warning("No metadata in backup")
    else:
        logger.info("No backup found. Run Cell 2 to extract.")

In [8]:
# [Cell 1.5] Item2Vec Vocab 곡 분석: 플레이리스트 등장 횟수 & 장르 분포

from gensim.models import Word2Vec

def analyze_vocab_distribution():
    """Item2Vec vocab에 포함된 곡들의 플레이리스트 등장 빈도 및 장르 분포 분석"""

    logger.info("Loading Item2Vec model...")
    model = Word2Vec.load(ITEM2VEC_MODEL_PATH)
    vocab_songs = set(model.wv.key_to_index.keys())
    logger.info(f"Total vocab songs: {len(vocab_songs)}")

    logger.info("Counting song appearances in playlists...")
    song_count = Counter()

    with open(TRAIN_JSON_PATH, 'r', encoding='utf-8') as f:
        train_data = json.load(f)

    for playlist in tqdm(train_data, desc="Processing playlists"):
        for song_id in playlist['songs']:
            song_id_str = str(song_id)
            if song_id_str in vocab_songs:
                song_count[song_id_str] += 1

    counts = list(song_count.values())
    print("\n" + "="*60)
    print("📊 플레이리스트 등장 횟수 분포 (Item2Vec Vocab 곡 기준)")
    print("="*60)
    print(f"분석 대상 곡 수: {len(counts):,}")
    print(f"최소 등장 횟수: {min(counts)}")
    print(f"최대 등장 횟수: {max(counts)}")
    print(f"평균 등장 횟수: {np.mean(counts):.2f}")
    print(f"중앙값: {np.median(counts):.2f}")

    print("\n[ 등장 횟수 구간별 분포 ]")
    bins = [0, 5, 10, 20, 50, 100, 200, 500, 1000, float('inf')]
    labels = ['1-5', '6-10', '11-20', '21-50', '51-100', '101-200', '201-500', '501-1000', '1000+']

    total = len(counts)
    for i in range(len(bins)-1):
        count_in_bin = sum(1 for c in counts if bins[i] < c <= bins[i+1])
        pct = (count_in_bin / total) * 100
        print(f"{labels[i]:>12}회: {count_in_bin:>6,}곡 ({pct:>5.2f}%)")

    logger.info("Loading song metadata and genre info...")
    with open(SONG_META_PATH, 'r', encoding='utf-8') as f:
        song_meta = json.load(f)

    with open(GENRE_JSON_PATH, 'r', encoding='utf-8') as f:
        genre_map = json.load(f)

    meta_dict = {str(s['id']): s for s in song_meta}

    genre_count = Counter()
    for song_id in vocab_songs:
        if song_id in meta_dict:
            genres = meta_dict[song_id].get('song_gn_gnr_basket', [])
            for genre_code in genres:
                genre_count[genre_code] += 1

    print("\n" + "="*60)
    print("🎵 장르별 분포 (상위 20개)")
    print("="*60)
    print(f"{'장르코드':<12} {'장르명':<30} {'곡 수':>10} {'비율':>8}")
    print("-"*60)

    for genre_code, count in genre_count.most_common(20):
        genre_name = genre_map.get(str(genre_code), "Unknown")
        pct = (count / len(vocab_songs)) * 100
        print(f"{str(genre_code):<12} {genre_name:<30} {count:>10,} {pct:>7.2f}%")

    print("\n💡 참고: 한 곡이 여러 장르를 가질 수 있어 합계가 100%를 초과할 수 있습니다.")
    print("="*60)

# 실행
analyze_vocab_distribution()

Processing playlists:   0%|          | 0/115071 [00:00<?, ?it/s]


📊 플레이리스트 등장 횟수 분포 (Item2Vec Vocab 곡 기준)
분석 대상 곡 수: 316,109
최소 등장 횟수: 2
최대 등장 횟수: 2175
평균 등장 횟수: 15.78
중앙값: 4.00

[ 등장 횟수 구간별 분포 ]
         1-5회: 191,483곡 (60.57%)
        6-10회: 49,548곡 (15.67%)
       11-20회: 32,969곡 (10.43%)
       21-50회: 24,172곡 ( 7.65%)
      51-100회:  9,446곡 ( 2.99%)
     101-200회:  4,841곡 ( 1.53%)
     201-500회:  2,843곡 ( 0.90%)
    501-1000회:    672곡 ( 0.21%)
       1000+회:    135곡 ( 0.04%)

🎵 장르별 분포 (상위 20개)
장르코드         장르명                                   곡 수       비율
------------------------------------------------------------
GN0100       발라드                                37,986   12.02%
GN0900       POP                                37,100   11.74%
GN1100       일렉트로니카                             26,417    8.36%
GN0500       인디음악                               26,296    8.32%
GN1000       록/메탈                               24,160    7.64%
GN1500       OST                                22,130    7.00%
GN1700       재즈                                 18,3

In [9]:
# [Cell 2] 데이터 준비: 등장 횟수 버킷 기반 계층 샘플링 & 메타데이터 기반 스킵

from gensim.models import Word2Vec

def assign_bucket(appear_cnt: int) -> str:
    """등장 횟수에 따라 버킷 할당"""
    if appear_cnt <= 5:
        return "B1"
    elif appear_cnt <= 10:
        return "B2"
    elif appear_cnt <= 20:
        return "B3"
    elif appear_cnt <= 50:
        return "B4"
    else:
        return "B5"

def load_song_stats_from_json() -> pd.DataFrame:
    """train.json에서 곡별 등장 횟수 통계 계산"""
    logger.info(f"Loading song stats from {TRAIN_JSON_PATH}...")

    with open(TRAIN_JSON_PATH, 'r', encoding='utf-8') as f:
        train_data = json.load(f)

    song_count = Counter()
    for playlist in tqdm(train_data, desc="Counting song appearances"):
        for song_id in playlist['songs']:
            song_count[song_id] += 1

    song_stats_df = pd.DataFrame([
        {"song_id": sid, "appear_cnt": cnt}
        for sid, cnt in song_count.items()
    ])

    logger.info(f"Loaded {len(song_stats_df):,} songs")
    return song_stats_df

def extract_needed_files_from_tar(tar_path: str, needed_members: List[str], extract_dir: str):
    """Tar 파일에서 필요한 파일만 추출"""
    os.makedirs(extract_dir, exist_ok=True)

    with tarfile.open(tar_path, 'r') as tar:
        members_to_extract = []
        for member in tar.getmembers():
            for needed in needed_members:
                if member.name.endswith(needed):
                    members_to_extract.append(member)
                    break

        if members_to_extract:
            tar.extractall(path=extract_dir, members=members_to_extract)
            logger.info(f"  Extracted {len(members_to_extract)} files from {os.path.basename(tar_path)}")
        else:
            logger.warning(f"  No matching files found in {os.path.basename(tar_path)}")

def prepare_stratified_train_ids(item2vec_path: str, tar_map_path: str) -> Tuple[List[int], Dict]:
    """등장 횟수 버킷 기반 계층 샘플링으로 train_song_ids 생성"""

    # 1. Item2Vec vocab 로드
    logger.info(f"Loading Item2Vec model from {item2vec_path}...")
    if not os.path.exists(item2vec_path):
        raise FileNotFoundError(f"Item2Vec model not found: {item2vec_path}")

    model = Word2Vec.load(item2vec_path)
    cf_song_ids = set(int(sid) for sid in model.wv.key_to_index.keys())
    logger.info(f"Item2Vec vocab size: {len(cf_song_ids):,}")

    # 2. Tar Map 로드
    logger.info(f"Loading Tar Map from {tar_map_path}...")
    if not os.path.exists(tar_map_path):
        raise FileNotFoundError(f"Tar map not found: {tar_map_path}")

    with open(tar_map_path, 'r') as f:
        tar_map = json.load(f)

    # 3. Mel 파일 존재 여부 확인
    valid_song_ids = []
    for sid in cf_song_ids:
        folder_id = str(sid // 1000)
        if folder_id in tar_map:
            valid_song_ids.append(sid)

    stage2_song_ids = set(valid_song_ids)
    logger.info(f"CF ∩ Mel intersection: {len(stage2_song_ids):,} songs")

    if not stage2_song_ids:
        raise ValueError("No intersection between CF vocab and Mel files!")

    # 4. 곡별 등장 횟수 통계 로드
    song_stats_df = load_or_create_song_stats()

    # 5. Stage2 교집합으로 필터링
    stage2_stats_df = song_stats_df[song_stats_df['song_id'].isin(stage2_song_ids)].copy()
    logger.info(f"Stage2 filtered stats: {len(stage2_stats_df):,} songs")

    if stage2_stats_df.empty:
        raise ValueError("Stage2 stats dataframe is empty after filtering!")

    # 6. 버킷 컬럼 추가
    stage2_stats_df['bucket'] = stage2_stats_df['appear_cnt'].apply(assign_bucket)

    bucket_counts = stage2_stats_df['bucket'].value_counts().to_dict()
    logger.info(f"Bucket distribution: {bucket_counts}")

    # 7. 버킷별 계층 샘플링
    train_song_ids = []
    actual_samples = {}

    rng = np.random.RandomState(RANDOM_SEED)

    for bucket_name, target_n in BUCKET_TARGET_COUNTS.items():
        bucket_df = stage2_stats_df[stage2_stats_df['bucket'] == bucket_name]
        available_n = len(bucket_df)
        sample_n = min(target_n, available_n)

        if available_n == 0:
            logger.warning(f"Bucket {bucket_name} is empty, skipping.")
            actual_samples[bucket_name] = 0
            continue

        if available_n < target_n:
            logger.warning(f"Bucket {bucket_name}: available {available_n} < target {target_n}")

        bucket_sample = bucket_df.sample(n=sample_n, random_state=RANDOM_SEED)
        sampled_ids = bucket_sample['song_id'].tolist()
        train_song_ids.extend(sampled_ids)
        actual_samples[bucket_name] = sample_n

        logger.info(f"  {bucket_name}: sampled {sample_n:,} / {target_n:,} (available: {available_n:,})")

    # 8. 후처리 및 검증
    train_song_ids = list(dict.fromkeys(train_song_ids))

    if len(train_song_ids) != len(set(train_song_ids)):
        logger.warning("Duplicates found in train_song_ids, removed.")

    rng.shuffle(train_song_ids)

    assert set(train_song_ids).issubset(stage2_song_ids), "train_song_ids not subset of stage2_song_ids!"

    logger.info(f"\n{'='*60}")
    logger.info("Stage2 train_song_ids 샘플링 완료")
    logger.info(f"  총 곡 수: {len(train_song_ids):,}")
    logger.info(f"  버킷별 실제 샘플 수: {actual_samples}")
    logger.info(f"{'='*60}\n")

    # 9. (Colab 전용) 메타데이터 기반 스마트 추출
    if IS_COLAB:
        logger.info("Checking extraction status and extracting needed files...")

        tar_to_songs: Dict[str, List[int]] = defaultdict(list)
        for sid in train_song_ids:
            folder_id = str(sid // 1000)
            if folder_id in tar_map:
                tar_name = tar_map[folder_id]
                tar_to_songs[tar_name].append(sid)

        logger.info(f"Required Tar files: {len(tar_to_songs)}")

        if not os.path.exists(DRIVE_MOUNT_PATH):
            from google.colab import drive
            drive.mount(DRIVE_MOUNT_PATH)

        os.makedirs(LOCAL_MEL_DIR, exist_ok=True)

        # 메타데이터 파일 경로
        metadata_path = os.path.join(LOCAL_MEL_DIR, "extraction_metadata.json")

        # 기존 메타데이터 로드
        if os.path.exists(metadata_path):
            with open(metadata_path, 'r') as f:
                extraction_metadata = json.load(f)
            logger.info(f"📂 Loaded extraction metadata: {len(extraction_metadata)} tar files already extracted")
        else:
            extraction_metadata = {}

        extracted_count = 0
        skipped_count = 0

        for tar_name, song_ids_in_tar in tqdm(tar_to_songs.items(), desc="Processing tar files"):
            tar_folder_name = tar_name.replace('.tar', '')
            extract_path = os.path.join(LOCAL_MEL_DIR, tar_folder_name)

            needed_song_ids = sorted(song_ids_in_tar)

            # 메타데이터 확인: 이미 같은 곡들을 추출했는지 체크
            if tar_name in extraction_metadata:
                extracted_song_ids = extraction_metadata[tar_name].get("song_ids", [])
                if sorted(extracted_song_ids) == needed_song_ids:
                    # 실제 파일 존재 여부도 샘플링 체크 (첫 5개)
                    sample_check = True
                    for sid in needed_song_ids[:min(5, len(needed_song_ids))]:
                        folder_id = str(sid // 1000)
                        file_path = os.path.join(extract_path, "arena_mel", folder_id, f"{sid}.npy")
                        if not os.path.exists(file_path):
                            sample_check = False
                            break

                    if sample_check:
                        skipped_count += 1
                        continue  # 이미 추출 완료, 스킵

            # 추출 필요
            needed_members = []
            for sid in song_ids_in_tar:
                folder_id = str(sid // 1000)
                needed_members.append(f"arena_mel/{folder_id}/{sid}.npy")

            tar_path = os.path.join(DRIVE_TAR_DIR, tar_name)
            if os.path.exists(tar_path):
                try:
                    extract_needed_files_from_tar(tar_path, needed_members, extract_path)

                    # 추출 성공 시 메타데이터 업데이트
                    extraction_metadata[tar_name] = {
                        "song_ids": needed_song_ids,
                        "extracted_at": pd.Timestamp.now().isoformat(),
                        "num_files": len(needed_song_ids)
                    }

                    # 메타데이터 즉시 저장 (중단되어도 이미 추출한 것은 보존)
                    with open(metadata_path, 'w') as f:
                        json.dump(extraction_metadata, f, indent=2)

                    extracted_count += 1

                except Exception as e:
                    logger.error(f"Failed to extract from {tar_name}: {e}")
            else:
                logger.warning(f"Tar file not found: {tar_path}")

        logger.info(f"✅ Extraction completed: {extracted_count} new, {skipped_count} skipped (already extracted)")
        logger.info(f"📁 Metadata saved to: {metadata_path}")

    return train_song_ids, tar_map

# 실행
train_song_ids, tar_map = prepare_stratified_train_ids(ITEM2VEC_MODEL_PATH, MELON_TAR_MAP_PATH)
logger.info(f"Final train_song_ids count: {len(train_song_ids):,}")

Counting song appearances:   0%|          | 0/115071 [00:00<?, ?it/s]

Processing tar files:   0%|          | 0/40 [00:00<?, ?it/s]

/tmp/ipython-input-385477767.py:68: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_dir, members=members_to_extract)


In [10]:
# [Cell 2.5] 추출된 파일을 Drive에 백업 (런타임 종료 대비)


if IS_COLAB:
    import shutil

    BACKUP_DIR = "/content/drive/MyDrive/Colab Notebooks/mel_data_backup"

    logger.info("Backing up extracted files to Drive...")

    try:
        os.makedirs(BACKUP_DIR, exist_ok=True)

        # 🔥 개선: 메타데이터 비교 후 필요한 것만 백업
        local_metadata = os.path.join(LOCAL_MEL_DIR, "extraction_metadata.json")
        backup_metadata = os.path.join(BACKUP_DIR, "extraction_metadata.json")

        if os.path.exists(local_metadata):
            # 메타데이터 복사 (항상 최신으로 유지)
            shutil.copy2(local_metadata, backup_metadata)
            logger.info("✅ Metadata backed up")

            # tar 폴더들 백업 (새로운 것만)
            with open(local_metadata, 'r') as f:
                metadata = json.load(f)

            for tar_name in metadata.keys():
                tar_folder = tar_name.replace('.tar', '')
                src = os.path.join(LOCAL_MEL_DIR, tar_folder)
                dst = os.path.join(BACKUP_DIR, tar_folder)

                if os.path.exists(src) and not os.path.exists(dst):
                    shutil.copytree(src, dst)
                    logger.info(f"  Backed up: {tar_folder}")

            logger.info(f"✅ Backup completed!")
        else:
            logger.warning("No metadata found, skipping backup")

    except Exception as e:
        logger.error(f"❌ Backup failed: {e}")

In [12]:
# [Cell 3] 멜 스펙 로더 (디렉토리 구조 반영)

def load_mel_spectrogram(song_id: int, base_dir: str = LOCAL_MEL_DIR, tar_map: Dict = tar_map) -> Optional[np.ndarray]:
    """
    song_id에 해당하는 mel npy 파일을 로드합니다.
    구조: {base_dir}/{tar_folder_name}/arena_mel/{folder_id}/{song_id}.npy
    """
    try:
        folder_id = str(song_id // 1000)
        if folder_id not in tar_map:
            return None

        tar_name = tar_map[folder_id]
        tar_folder_name = tar_name.replace('.tar', '')

        # 미리 추출된 파일에서 로드
        path = os.path.join(base_dir, tar_folder_name, "arena_mel", folder_id, f"{song_id}.npy")

        if not os.path.exists(path):
            return None

        mel = np.load(path)

        # (48, T) 형태 확인 및 Transpose
        if mel.shape[0] != 48 and mel.shape[1] == 48:
            mel = mel.T
        return mel

    except Exception as e:
        return None

def random_crop(mel: np.ndarray, fixed_t: int = FIXED_T) -> np.ndarray:
    """멜 스펙트로그램을 fixed_t 길이로 랜덤 크롭하거나 패딩합니다."""
    n_freq, n_time = mel.shape

    if n_time >= fixed_t:
        start = random.randint(0, n_time - fixed_t)
        return mel[:, start:start+fixed_t]
    else:
        pad_total = fixed_t - n_time
        pad_left = pad_total // 2
        pad_right = pad_total - pad_left
        return np.pad(mel, ((0, 0), (pad_left, pad_right)), mode='constant', constant_values=0)

def augment_mel(mel: np.ndarray) -> np.ndarray:
    """간단한 데이터 증강"""
    aug_mel = mel.copy()
    if random.random() < 0.5:
        noise = np.random.normal(0, 0.005, aug_mel.shape)
        aug_mel += noise
    if random.random() < 0.5:
        shift = random.randint(-5, 5)
        if shift != 0:
            aug_mel = np.roll(aug_mel, shift, axis=1)
    return aug_mel

def mel_to_tensor(mel: np.ndarray) -> torch.Tensor:
    return torch.from_numpy(mel).unsqueeze(0).float()

NameError: name 'tar_map' is not defined

In [12]:
# [Cell 4] Contrastive Dataset & DataLoader 정의

class ContrastiveMelDataset(data.Dataset):
    def __init__(self, song_ids: List[int], mel_dir: str = LOCAL_MEL_DIR, tar_map: Dict = None):
        self.song_ids = song_ids
        self.mel_dir = mel_dir
        self.tar_map = tar_map if tar_map is not None else {}

    def __len__(self):
        return len(self.song_ids)

    def __getitem__(self, index):
        song_id = self.song_ids[index]

        mel = load_mel_spectrogram(song_id, self.mel_dir, self.tar_map)

        if mel is None:
            mel = np.zeros((48, FIXED_T))

        mel1 = random_crop(mel, FIXED_T)
        mel2 = random_crop(mel, FIXED_T)

        mel1 = augment_mel(mel1)
        mel2 = augment_mel(mel2)

        return {
            "song_id": song_id,
            "anchor": mel_to_tensor(mel1),
            "positive": mel_to_tensor(mel2)
        }

def create_dataloader(song_ids: List[int], tar_map: Dict, batch_size: int = BATCH_SIZE):
    dataset = ContrastiveMelDataset(song_ids, tar_map=tar_map)
    return data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        drop_last=True
    )

In [13]:
# [Cell 5] 소형 멜-CNN 인코더 모델 정의

class MelCNNEncoder(nn.Module):
    def __init__(self, embed_dim: int = EMBED_DIM):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, embed_dim),
            nn.LayerNorm(embed_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.gap(x)
        x = self.fc(x)
        x = F.normalize(x, p=2, dim=-1)
        return x

model = MelCNNEncoder(EMBED_DIM).to(device)
logger.info(model)

In [14]:
# [Cell 6] InfoNCE / SimCLR 스타일 Contrastive Loss 구현

def info_nce_loss(z_i: torch.Tensor, z_j: torch.Tensor, temperature: float = TEMPERATURE) -> torch.Tensor:
    """
    z_i, z_j: (batch_size, embed_dim) 형태의 L2 normalized 임베딩.
    같은 인덱스끼리 positive 쌍, 나머지는 negative로 보는 InfoNCE loss.
    """
    batch_size = z_i.shape[0]
    z = torch.cat([z_i, z_j], dim=0)
    sim = torch.matmul(z, z.T) / temperature

    labels = torch.cat([
        torch.arange(batch_size, 2 * batch_size, device=z.device),
        torch.arange(0, batch_size, device=z.device)
    ], dim=0)

    mask = torch.eye(2 * batch_size, device=z.device).bool()
    sim.masked_fill_(mask, -9e15)

    loss = F.cross_entropy(sim, labels)
    return loss

In [15]:
# [Cell 7] 학습 루프 (Training Loop)

def train_model():
    if not train_song_ids:
        logger.warning("No training songs found. Skipping training.")
        return

    set_seed(RANDOM_SEED)

    train_loader = create_dataloader(train_song_ids, tar_map, BATCH_SIZE)
    logger.info(f"Starting training for {NUM_EPOCHS} epochs...")

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

    for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0.0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        for batch in pbar:
            anchor = batch['anchor'].to(device)
            positive = batch['positive'].to(device)

            z_i = model(anchor)
            z_j = model(positive)

            loss = info_nce_loss(z_i, z_j, TEMPERATURE)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(train_loader)
        logger.info(f"Epoch {epoch+1} done. Avg Loss: {avg_loss:.4f}")

# 학습 실행
train_model()

Epoch 1/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 2/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 3/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 4/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 5/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 6/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 7/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 8/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 9/10:   0%|          | 0/781 [00:00<?, ?it/s]

Epoch 10/10:   0%|          | 0/781 [00:00<?, ?it/s]

In [17]:
# [Cell 8] 곡 임베딩 추출 함수 정의 (chunk 단위 처리 가능하게)

def extract_embeddings_for_song_ids(
    model: MelCNNEncoder,
    song_ids: List[int],
    output_path: str,
    tar_map: Dict,
    mel_dir: str = LOCAL_MEL_DIR
) -> None:
    """
    주어진 song_id 리스트에 대해 멜 스펙을 로드하고,
    학습된 인코더로 임베딩을 추출해 .npz로 저장합니다.
    """
    model.eval()
    emb_dict: Dict[int, np.ndarray] = {}

    logger.info(f"Extracting embeddings for {len(song_ids)} songs...")

    with torch.no_grad():
        for song_id in tqdm(song_ids):
            mel = load_mel_spectrogram(song_id, mel_dir, tar_map)
            if mel is None:
                continue

            mel_crop = random_crop(mel, FIXED_T)
            tensor = mel_to_tensor(mel_crop).to(device)
            z = model(tensor.unsqueeze(0))
            emb_dict[song_id] = z.squeeze(0).cpu().numpy()

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    np.savez_compressed(output_path, **{str(k): v for k, v in emb_dict.items()})
    logger.info(f"Saved embeddings to {output_path} (Count: {len(emb_dict)})")

In [18]:
# [Cell 9] 임베딩 추출 함수 예시 호출 (train_song_ids 대상으로)

if train_song_ids:
    extract_embeddings_for_song_ids(
        model=model,
        song_ids=train_song_ids,
        output_path=STAGE2_EMB_OUTPUT_PATH,
        tar_map=tar_map
    )

  0%|          | 0/50000 [00:00<?, ?it/s]

In [3]:
# [Cell 10] 간단 Sanity Check (이웃 곡 확인)

def sanity_check_neighbors(emb_path: str, meta_path: str = SONG_META_PATH):
    if not os.path.exists(emb_path):
        logger.warning("Embedding file not found. Skipping sanity check.")
        return

    data = np.load(emb_path)
    keys = list(data.keys())
    vectors = np.array([data[k] for k in keys])

    logger.info(f"Loaded {len(keys)} embeddings for sanity check.")

    if not os.path.exists(meta_path):
        logger.warning(f"Meta file not found at {meta_path}")
        return

    try:
        with open(meta_path, 'r', encoding='utf-8') as f:
            song_meta = json.load(f)
        meta_dict = {str(s['id']): s for s in song_meta}
    except Exception as e:
        logger.warning(f"Failed to load song_meta: {e}")
        meta_dict = {}

    n_seeds = 3
    seeds = random.sample(keys, min(len(keys), n_seeds))

    for seed_id in seeds:
        seed_vec = data[seed_id]
        sims = np.dot(vectors, seed_vec)
        top_k_idx = np.argsort(sims)[::-1][:6]

        print(f"\n[Seed Song] {meta_dict.get(seed_id, {}).get('song_name', seed_id)} / {meta_dict.get(seed_id, {}).get('artist_name_basket', [])}")
        print("-" * 40)

        for idx in top_k_idx:
            t_id = keys[idx]
            t_score = sims[idx]
            t_meta = meta_dict.get(t_id, {})
            print(f"{t_score:.4f} | {t_meta.get('song_name', t_id)} | {t_meta.get('artist_name_basket', [])} | {t_meta.get('gnr_basket', [])}")

# 실행
sanity_check_neighbors(STAGE2_EMB_OUTPUT_PATH)

INFO:__main__:Loaded 50000 embeddings for sanity check.



[Seed Song] Summer Luv (feat. Crystal Fighters) / ['Whethan', 'The Knocks']
----------------------------------------
1.0000 | Summer Luv (feat. Crystal Fighters) | ['Whethan', 'The Knocks'] | []
0.7520 | Poolside | ['Baynk'] | []
0.7407 | The Fall (Maurice Fulton Remix) | ['Rhye'] | []
0.7377 | Keep Us Alive | ['Josh Tobias'] | []
0.7301 | Day And Night | ['Lo Air'] | []
0.7021 | Rainy Blue (Feat. Yebin) | ['PS영준'] | []

[Seed Song] Skit / ['Dulce']
----------------------------------------
1.0000 | Skit | ['Dulce'] | []
0.8189 | Waltz For Red Sarah | ['척추'] | []
0.7704 | Ann`s Tune | ['Avishai Cohen'] | []
0.7229 | Longing For You | ['Yoshiko Kishino'] | []
0.7046 | Brahms : 4 Piano Pieces No.4 In E Flat Major Op.119 - Rhapsodie (브람스 : 4개의 피아노 소품 4번 내림 마장조 작품번호 119 - 광시곡) | ['피아노클래식'] | []
0.6929 | Deep River | ['Hank Jones', 'Charlie Haden'] | []

[Seed Song] 내가 아까워 / ['아이스 (I.C.E)']
----------------------------------------
1.0000 | 내가 아까워 | ['아이스 (I.C.E)'] | []
0.7940 | Reflex | ['인

In [5]:
# [Cell 10.5] 수정 버전 - 임베딩에 실제로 있는 곡으로 추천

def search_songs_in_embeddings(keyword: str, emb_path: str, meta_path: str, max_results: int = 5):
    """임베딩에 실제로 있는 곡 검색"""

    # 임베딩 로드
    data = np.load(emb_path)
    available_ids = set(data.keys())

    # 메타데이터 로드
    with open(meta_path, 'r', encoding='utf-8') as f:
        song_meta = json.load(f)

    # 검색
    results = []
    for song in song_meta:
        sid = str(song['id'])
        if sid in available_ids:
            song_name = song.get('song_name', '')
            artists = song.get('artist_name_basket', [])

            # 키워드 매칭
            if keyword.lower() in song_name.lower() or \
               any(keyword.lower() in artist.lower() for artist in artists):
                results.append({
                    'id': sid,
                    'name': song_name,
                    'artists': artists,
                    'genres': song.get('song_gn_gnr_basket', [])
                })

    return results[:max_results]

def recommend_by_song(seed_song_id: str, emb_path: str, meta_path: str, top_k: int = 10):
    """특정 곡 기반으로 추천"""

    data = np.load(emb_path)
    keys = list(data.keys())
    vectors = np.array([data[k] for k in keys])

    with open(meta_path, 'r', encoding='utf-8') as f:
        song_meta = json.load(f)
    meta_dict = {str(s['id']): s for s in song_meta}

    if seed_song_id not in keys:
        print(f"❌ Song ID {seed_song_id} not in embeddings")
        return

    seed_meta = meta_dict.get(seed_song_id, {})
    print(f"\n{'='*70}")
    print(f"🎵 SEED: {seed_meta.get('song_name', seed_song_id)}")
    print(f"   Artist: {', '.join(seed_meta.get('artist_name_basket', []))}")
    print(f"   Genre: {seed_meta.get('song_gn_gnr_basket', [])}")
    print(f"{'='*70}\n")

    seed_vec = data[seed_song_id]
    sims = np.dot(vectors, seed_vec)
    top_k_idx = np.argsort(sims)[::-1][:top_k]

    print(f"📋 Top {top_k} Recommendations:\n")
    for rank, idx in enumerate(top_k_idx, 1):
        rec_id = keys[idx]
        score = sims[idx]
        rec_meta = meta_dict.get(rec_id, {})

        print(f"{rank:2d}. [{score:.4f}] {rec_meta.get('song_name', rec_id)}")
        print(f"     👤 {', '.join(rec_meta.get('artist_name_basket', []))}")
        print(f"     🎸 {rec_meta.get('song_gn_gnr_basket', [])}")
        print()

# ==========================================
# 🎵 오디오 인코더 기반 추천 (Seed 입력 방식)
# ==========================================

# 🎯 여기에 원하는 seed_song_id를 입력하세요
SEED_SONG_ID = 290707  # 예: 261 (아이유 - 을의 연애)

# 곡 검색 헬퍼 (song_id를 모를 때 사용)
# search_songs_in_embeddings("아티스트명 또는 곡명", STAGE2_EMB_OUTPUT_PATH, SONG_META_PATH)

if SEED_SONG_ID is not None:
    # Seed가 지정된 경우 추천 실행
    recommend_by_song(str(SEED_SONG_ID), STAGE2_EMB_OUTPUT_PATH, SONG_META_PATH)
else:
    print("⚠️ SEED_SONG_ID를 설정해주세요!")
    print("   예: SEED_SONG_ID = 261")
    print("\n💡 곡 검색이 필요하면 아래 코드를 주석 해제해서 실행하세요:")
    print("   search_songs_in_embeddings('아이유', STAGE2_EMB_OUTPUT_PATH, SONG_META_PATH)")
    print("\n📋 검색 예시 결과:")
    # 예시로 몇 가지 아티스트 검색 결과 출력
    for artist in ["아이유", "잔나비"]:
        results = search_songs_in_embeddings(artist, STAGE2_EMB_OUTPUT_PATH, SONG_META_PATH, max_results=3)
        if results:
            print(f"\n  🔍 '{artist}' 검색 결과:")
            for song in results:
                print(f"     - {song['name']} (ID: {song['id']})")


🎵 SEED: CHANGE BY CHANGE
   Artist: T-Square
   Genre: ['GN1700']

📋 Top 10 Recommendations:

 1. [1.0000] CHANGE BY CHANGE
     👤 T-Square
     🎸 ['GN1700']

 2. [0.7341] 사랑여우별
     👤 아름드리
     🎸 ['GN2400']

 3. [0.7072] Mr.Clean
     👤 Marcus Miller
     🎸 ['GN1700']

 4. [0.7067] No One In The World
     👤 Anita Baker
     🎸 ['GN1300']

 5. [0.7037] El Toro Relajo (The Troublesome Bull) (2016 Remastered)
     👤 Linda Ronstadt
     🎸 ['GN0900']

 6. [0.6933] 산타할아버지 우리마을에 오시네
     👤 유리상자
     🎸 ['GN0100']

 7. [0.6923] Call Me (Spanish Version)
     👤 Blondie
     🎸 ['GN0900']

 8. [0.6892] Pee Wee
     👤 Candy Dulfer
     🎸 ['GN1700']

 9. [0.6833] 나의 등 뒤에서 + 곤한 내 영혼 편히 쉴곳과
     👤 브라운 워십
     🎸 ['GN2100']

10. [0.6775] Crawl
     👤 Chris Brown
     🎸 ['GN1300']



In [9]:
# [새 셀 추가] 멜 스펙트로그램 시간 축 길이 분포 확인

import matplotlib.pyplot as plt
from collections import Counter

def analyze_mel_time_lengths(sample_size=1000):
    """
    멜 스펙트로그램의 시간 축 길이 분포를 분석합니다.
    
    Args:
        sample_size: 분석할 곡 수 (전체 분석 시 None)
    """
    time_lengths = []
    missing_count = 0
    
    # 샘플링할 곡 ID 리스트 (전체 또는 샘플)
    if sample_size:
        # train.json에서 샘플링
        with open(TRAIN_JSON_PATH, 'r', encoding='utf-8') as f:
            train_data = json.load(f)
        
        all_song_ids = set()
        for playlist in train_data:
            all_song_ids.update(playlist['songs'])
        
        sample_ids = random.sample(list(all_song_ids), min(sample_size, len(all_song_ids)))
    else:
        # 전체 분석 (느림)
        sample_ids = list(all_song_ids)
    
    logger.info(f"📊 {len(sample_ids)}개 곡의 멜 스펙트로그램 길이 분석 중...")
    
    for song_id in tqdm(sample_ids):
        mel = load_mel_spectrogram(song_id, LOCAL_MEL_DIR, tar_map)
        if mel is None:
            missing_count += 1
            continue
        
        n_freq, n_time = mel.shape
        time_lengths.append(n_time)
    
    if not time_lengths:
        logger.warning("❌ 분석할 수 있는 멜 스펙트로그램이 없습니다.")
        return
    
    time_lengths = np.array(time_lengths)
    
    # 통계 출력
    print("\n" + "="*70)
    print("📊 멜 스펙트로그램 시간 축 길이 통계")
    print("="*70)
    print(f"  총 분석 곡 수: {len(time_lengths)}")
    print(f"  누락된 곡 수: {missing_count}")
    print(f"\n  최소값: {time_lengths.min()}")
    print(f"  최대값: {time_lengths.max()}")
    print(f"  평균: {time_lengths.mean():.1f}")
    print(f"  중앙값: {np.median(time_lengths):.1f}")
    print(f"  표준편차: {time_lengths.std():.1f}")
    
    # FIXED_T = 256 기준 분석
    print(f"\n  FIXED_T = 256 기준:")
    shorter = (time_lengths < 256).sum()
    longer = (time_lengths >= 256).sum()
    print(f"    - 256보다 짧은 곡: {shorter}개 ({shorter/len(time_lengths)*100:.1f}%) → 패딩 필요")
    print(f"    - 256보다 긴 곡: {longer}개 ({longer/len(time_lengths)*100:.1f}%) → 크롭 필요")
    
    # 분위수
    print(f"\n  분위수:")
    for q in [10, 25, 50, 75, 90, 95, 99]:
        val = np.percentile(time_lengths, q)
        print(f"    {q}%: {val:.1f}")
    
    # 히스토그램
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(time_lengths, bins=50, edgecolor='black', alpha=0.7)
    plt.axvline(256, color='red', linestyle='--', linewidth=2, label='FIXED_T=256')
    plt.xlabel('시간 축 길이 (프레임 수)')
    plt.ylabel('곡 수')
    plt.title('멜 스펙트로그램 시간 축 길이 분포')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.boxplot(time_lengths, vert=True)
    plt.axhline(256, color='red', linestyle='--', linewidth=2, label='FIXED_T=256')
    plt.ylabel('시간 축 길이 (프레임 수)')
    plt.title('박스플롯')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 권장값 제안
    median = np.median(time_lengths)
    p25 = np.percentile(time_lengths, 25)
    p75 = np.percentile(time_lengths, 75)
    
    print(f"\n💡 권장 FIXED_T 값:")
    print(f"   - 현재: 256")
    print(f"   - 중앙값 기반: {int(median)}")
    print(f"   - 25% 분위수: {int(p25)} (짧은 곡 고려)")
    print(f"   - 75% 분위수: {int(p75)} (긴 곡 고려)")
    
    if median < 256:
        print(f"\n   ⚠️ 중앙값({int(median)})이 256보다 작습니다.")
        print(f"      → 많은 곡이 패딩되어 정보 손실 가능")
    elif median > 256 * 2:
        print(f"\n   ⚠️ 중앙값({int(median)})이 256의 2배보다 큽니다.")
        print(f"      → 많은 곡이 크롭되어 정보 손실 가능")
    else:
        print(f"\n   ✅ 현재 FIXED_T=256이 적절해 보입니다!")

# 실행 (샘플 1000개 분석)
analyze_mel_time_lengths(sample_size=1000)

INFO:__main__:📊 1000개 곡의 멜 스펙트로그램 길이 분석 중...


  0%|          | 0/1000 [00:00<?, ?it/s]

NameError: name 'load_mel_spectrogram' is not defined